# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files("data/song_data")

In [5]:
filepath = song_files[0]

In [6]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD0S291187B9B7BF5,NaN,Ohio,NaN,Rated R,114.78159,1,SOMJBYD12A6D4F8557,Keepin It Real (Skit),0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
song_data = df[["song_id", "title", "artist_id", "year", "duration"]].values.tolist()[0]
song_data

['SOMJBYD12A6D4F8557',
 'Keepin It Real (Skit)',
 'ARD0S291187B9B7BF5',
 0,
 114.78159]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
artist_data = df[["artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"]].values.tolist()[0]
artist_data

['ARD0S291187B9B7BF5', 'Rated R', 'Ohio', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [10]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [11]:
log_files = get_files("data/log_data")

In [12]:
filepath = log_files[0]

In [13]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Dee Dee Bridgewater,Logged In,Lily,F,38,Koch,318.64118,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,La Vie En Rose,200,1542845032796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
1,Tim O'brien,Logged In,Lily,F,39,Koch,176.14322,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,Think About Last Night,200,1542845350796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
2,Nirvana,Logged In,Lily,F,40,Koch,215.11791,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,Love Buzz,200,1542845526796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
3,Weezer,Logged In,Lily,F,41,Koch,479.32036,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,Only In Dreams,200,1542845741796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
4,Nightwish,Logged In,Lily,F,42,Koch,286.19710,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,She Is My Sin,200,1542846220796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [14]:
df = df[df["page"]=="NextSong"]
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Dee Dee Bridgewater,Logged In,Lily,F,38,Koch,318.64118,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,La Vie En Rose,200,1542845032796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
1,Tim O'brien,Logged In,Lily,F,39,Koch,176.14322,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,Think About Last Night,200,1542845350796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
2,Nirvana,Logged In,Lily,F,40,Koch,215.11791,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,Love Buzz,200,1542845526796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
3,Weezer,Logged In,Lily,F,41,Koch,479.32036,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,Only In Dreams,200,1542845741796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15
4,Nightwish,Logged In,Lily,F,42,Koch,286.19710,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,818,She Is My Sin,200,1542846220796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15


In [15]:
t = df["ts"]
t.head()

0    1542845032796
1    1542845350796
2    1542845526796
3    1542845741796
4    1542846220796
Name: ts, dtype: int64

In [16]:
datetime = pd.to_datetime(t, unit="ms")
time_data = (datetime, datetime.dt.hour, datetime.dt.day, datetime.dt.weekofyear, datetime.dt.month, datetime.dt.year, datetime.dt.weekday)
column_labels = ("timestamp", "hour", "day", "week of year", "month", "year", "weekday")

In [17]:
time_df = pd.DataFrame.from_dict(dict(zip(column_labels, time_data))) 
time_df.head()

,timestamp,hour,day,week of year,month,year,weekday
0,2018-11-22 00:03:52.796,0,22,47,11,2018,3
1,2018-11-22 00:09:10.796,0,22,47,11,2018,3
2,2018-11-22 00:12:06.796,0,22,47,11,2018,3
3,2018-11-22 00:15:41.796,0,22,47,11,2018,3
4,2018-11-22 00:23:40.796,0,22,47,11,2018,3


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [18]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [19]:
user_df = df[["userId", "firstName", "lastName", "gender", "level"]]

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [20]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [21]:
for index, row in df.iterrows():
    # get songid and artistid from song and artist tables
    print(row.song, row.artist, row.length)
    cur.execute(song_select,(row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    datetime = pd.to_datetime(row.ts, unit="ms")
    songplay_data = (datetime, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

La Vie En Rose Dee Dee Bridgewater 318.64118
Think About Last Night Tim O'brien 176.14322
Love Buzz Nirvana 215.11791
Only In Dreams Weezer 479.32036
She Is My Sin Nightwish 286.1971
Teach Me How To Dougie California Swag District 239.17669
Silvia Miike Snow 385.35791
I Kissed A Girl Katy Perry 179.40853
Peep Show Sikth 250.53995
Not Fair Lily Allen 199.88853
Lump The Presidents of the United States of America 495.77751
Right Now (Produced by Ayatollah) Wordsworth 253.1522
Take A Bow Rihanna 229.04118
Back To The African Garden Tomas Bodin 396.53832
My Humps Black Eyed Peas 326.86975
Show Me What I'm Looking For (Album Version) Carolina Liar 240.45669
Dust in The Wind Kansas 202.29179
Xehasmeni Melodia Onar 306.6771
Lakini's Juice Live 286.98077
Nuff Fire Abstract Rude 196.85832
Mean Mean Son Of A Gun Johnny Horton 131.81342
Ironmasters The Men They Couldn't Hang 251.14077
The Absence Of God (Album Version) Rilo Kiley 234.03057
Lost My Mind Shwayze 201.63873
Mamma Bram Vermeulen 251.42

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [22]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.